<a href="https://colab.research.google.com/github/Adhira-Deogade/TextSummarization_PointerGenerator/blob/master/binsss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate


In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [4]:
ls

adc.json  Drive/  My/  sample_data/


In [5]:
cd Drive/TS/TS/stanford-corenlp-full-2018-10-05/

/content/Drive/TS/TS/stanford-corenlp-full-2018-10-05


In [6]:
ls

 build.xml                                  LICENSE.txt
 cnn_stories_tokenized/                     make_datafiles3.py
 corenlp.sh                                 Makefile
 CoreNLP-to-HTML.xsl                        patterns/
 dm_stories_tokenized/                      pom-java-11.xml
 ejml-0.23.jar                              pom.xml
 ejml-0.23-src.zip                          protobuf.jar
 finished_files/                            README.txt
 input.txt                                  RESOURCE-LICENSES
 input.txt.xml                              SemgrexDemo.java
 javax.activation-api-1.2.0.jar             ShiftReduceDemo.java
 javax.activation-api-1.2.0-sources.jar     slf4j-api.jar
 javax.json-api-1.0-sources.jar             slf4j-simple.jar
 javax.json.jar                             stanford-corenlp-3.9.2.jar
 jaxb-api-2.4.0-b180830.0359.jar            stanford-corenlp-3.9.2-javadoc.jar
 jaxb-api-2.4.0-b180830.0359-sources.jar    stanford-corenlp-3.9.2-models.jar
 jaxb-core-2.3.

In [0]:

import os, os.path, random
import subprocess
import hashlib
import struct
import collections
import tensorflow as tf
from tensorflow.core.example import example_pb2
from stanfordcorenlp import StanfordCoreNLP


nlp = StanfordCoreNLP(r'/content/Drive/TS/TS/stanford-corenlp-full-2018-10-05')

In [0]:
def split_data(stories_dir,train=0.5,test=0.3):
    val = 1-(train+test)
    
    all_files = os.listdir(stories_dir)
    total = len(all_files)
    shuffled = all_files[:]
    random.shuffle(shuffled)
    
    train_stories = int(round(train*total))
    test_stories = int(round(test*total))
    val_stories = int(round(val*total))

    train_data = shuffled[:train_stories]
    test_data = shuffled[train_stories:train_stories+test_stories]
    val_data = shuffled[train_stories+test_stories:train_stories+test_stories+val_stories]
    return train_data,test_data,val_data


In [0]:
def tokenize_stories(stories_dir, tokenized_stories_dir):
    """Maps a whole directory of .story files to a tokenized version using Stanford CoreNLP Tokenizer"""
    print(f"Preparing to tokenize {stories_dir} to {tokenized_stories_dir}...") 
    stories = os.listdir(stories_dir)
    # make IO list file
    print("Making list of files to tokenize...")
    with open("mapping.txt", "w") as f:
        for s in stories:
            f.write("%s \t %s\n" % (os.path.join(stories_dir, s), os.path.join(tokenized_stories_dir, s)))
    
    command = ['java','-cp', "*", '-Xmx2g', 'edu.stanford.nlp.process.PTBTokenizer', '-ioFileList', '-preserveLines', 'mapping.txt']
    print(f"Tokenizing {len(stories)} files in {stories_dir} and saving in {tokenized_stories_dir}...")
    subprocess.call(command)
    
    print("Stanford CoreNLP Tokenizer has finished.")
    
    os.remove("mapping.txt")

    # Check that the tokenized stories directory contains the same number of files as the original directory
    num_orig = len(os.listdir(stories_dir))
    num_tokenized = len(os.listdir(tokenized_stories_dir))
    print (f"Successfully finished tokenizing {stories_dir} to {tokenized_stories_dir}.\n")

In [0]:
dm_single_close_quote = u'\u2019' # unicode
dm_double_close_quote = u'\u201d'
END_TOKENS = ['.', '!', '?', '...', "'", "`", '"', dm_single_close_quote, dm_double_close_quote, ")"] # acceptable ways to end a sentence
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'
VOCAB_SIZE = 2000
CHUNK_SIZE = 10


def read_text_file(text_file):
    lines = []
    with open(text_file, "r") as f:
        for line in f:
            lines.append(line.strip())
    return lines

def fix_missing_period(line):
    """Adds a period to a line that is missing a period"""
    if "@highlight" in line: return line
    if line=="": return line
    if line[-1] in END_TOKENS: return line
    return line + " ."


def get_art_abs(story_file):
    lines = read_text_file(story_file)

    # Lowercase everything
    lines = [line.lower() for line in lines]

    # Put periods on the ends of lines that are missing them (this is a problem in the dataset because many image captions don't end in periods; consequently they end up in the body of the article as run-on sentences)
    lines = [fix_missing_period(line) for line in lines]

    # Separate out article and abstract sentences
    article_lines = []
    highlights = []
    next_is_highlight = False
   
    for idx,line in enumerate(lines):
        if line == "":
            continue # empty line
        elif line.startswith("@highlight"):
            next_is_highlight = True
        elif next_is_highlight:
            highlights.append(line)
        else:
            article_lines.append(line)

        # Make article into a single string
        article = ' '.join(article_lines)

        # Make abstract into a signle string, putting <s> and </s> tags around the sentences
        abstract = ' '.join(["%s %s %s" % (SENTENCE_START, sent, SENTENCE_END) for sent in highlights])

    return article, abstract

In [0]:
def write_to_bin(file_names,story_dir,out_file,finished_files_dir, makevocab=False):
    """Reads the tokenized .story files corresponding to the names listed in the url_file and writes them to an out_file."""
    
    story_fnames = file_names
    num_stories = len(story_fnames)

    if makevocab:
        vocab_counter = collections.Counter()

    with open(out_file, 'wb') as writer:
        for idx,s in enumerate(story_fnames):
            if idx % 1000 == 0:
                print (f"Writing story {idx} of {num_stories}; {float(idx)*100.0/float(num_stories)} percent done")

            story_file = os.path.join(story_dir, s)
            
            # Get the strings to write to .bin file
            article, abstract = get_art_abs(story_file)

            # Write to tf.Example
            if bytes(article, 'utf-8') == 0:
                print('error!')
                
            tf_example = example_pb2.Example()
            tf_example.features.feature['article'].bytes_list.value.extend([bytes(article, 'utf-8')])
            tf_example.features.feature['abstract'].bytes_list.value.extend([bytes(abstract, 'utf-8')])
            tf_example_str = tf_example.SerializeToString()
            str_len = len(tf_example_str)
            writer.write(struct.pack('q', str_len))
            writer.write(struct.pack('%ds' % str_len, tf_example_str))

            # Write the vocab to file, if applicable
            if makevocab:
                art_tokens = article.split(' ')
                abs_tokens = abstract.split(' ')
                abs_tokens = [t for t in abs_tokens if t not in [SENTENCE_START, SENTENCE_END]] # remove these tags from vocab
                tokens = art_tokens + abs_tokens
                tokens = [t.strip() for t in tokens] # strip
                tokens = [t for t in tokens if t!=""] # remove empty
                vocab_counter.update(tokens)

    print(f"Finished writing file {out_file}\n") 
    # write vocab to file
    if makevocab:
        print ("Writing vocab file...")
        with open(os.path.join(finished_files_dir, "vocab"), 'w') as writer:
            for word, count in vocab_counter.most_common(VOCAB_SIZE):
                writer.write(word + ' ' + str(count) + '\n')
        print ("Finished writing vocab file" )

In [13]:
pwd

'/content/Drive/TS/TS/stanford-corenlp-full-2018-10-05'

In [0]:
mkdir dataFiles

In [15]:
cd dataFiles

/content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/dataFiles


In [0]:
mkdir  test_tokenized test_processed

In [0]:
stories_dir = "/content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/TotalStories"
tokenized_stories_dir = "/content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/TotalTokenized"
processed_dir = "//content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/dataFiles/test_processed"

In [0]:
train,test,val = split_data(stories_dir)

In [31]:
test

['35f0e33de7923036a97ac245d899f990bda5e242.story',
 '55a6aabd120f0b18297f7efe22452f9b3aca0d8d.story',
 '7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a.story',
 '0a0ae32d5581f71acb87e191a98ac27b497b232e.story',
 '42c027e4ff9730fbb3de84c1af0d2c506e41c3e4.story',
 '7c0e61ac829a3b3b653e2e3e7536cc4881d1f264.story',
 '30cb674030b01014d2c87b92f09152857e249631.story',
 '0a0b1eeeeb4f2179293e39d36b6fac0cef271c40.story',
 '0a0bd6bed053b3bcc3414c3ef29e4aadaaf3d5b0.story']

In [32]:
train

['017d27d00eb43678c15cb4a8dd4723a035323219.story',
 '77d7c8cf2a9432e395d629371a12790c563c19f7.story',
 '0a0b3d0b0ab9ee056856cdb241a58fc1ea049d53.story',
 '7fa8d9beca50155fd3d71a9087a4311ce321c34b.story',
 '9d7fc7fd9ffbfecb1c458b9929fa02e6c3538368.story',
 '0a0a007398ab02d4bd3c1ef9e034f567afa76b2c.story',
 '0a0b2b48245b8e585fed35eb89f69a8b1f0ee25a.story',
 '0a0afa9106783f8b7aca75d53054b51ca7293fb2.story',
 '2f43e9dfaa43ffbddbce339a8b6403ddce43b38a.story',
 '2ad31cae96512af5105b9b23f9b681dc732b2605.story',
 '0d43b97000ff852282c89d8d105e41495c0ee9bd.story',
 '0a0ad513d8b3f9ae5afd2d1caeb9d85c6236fa66.story',
 '06bc2f0272d033b123e9eebbb221c62d7d51c283.story',
 '0a0ad50bb4952bdfca93ff74565f1ac80e740278.story',
 '0a0a733db965c3fdf9bc2895104a1ef884a3d593.story']

In [33]:
val

['0a0a09d3e5a0f04d792a7d950af976ed7f1a2de7.story',
 '0a0a1a0e94aac65f2b50815050dfaccf521dda35.story',
 '0a0ba0e482cb72efffc0710e5479659fc23d346d.story',
 '0a0beabdef4146beb1171dd9f927ad70c2a08317.story',
 '5e22bbfc7232418b8d2dd646b952e404df5bd048.story',
 '0a00a9aebcb754c51534867cf1db2335dcb76884.story']

In [34]:
tokenize_stories(stories_dir, tokenized_stories_dir)

Preparing to tokenize /content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/TotalStories to /content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/TotalTokenized...
Making list of files to tokenize...
Tokenizing 30 files in /content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/TotalStories and saving in /content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/TotalTokenized...
Stanford CoreNLP Tokenizer has finished.
Successfully finished tokenizing /content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/TotalStories to /content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/TotalTokenized.



In [0]:
train_out_file = os.path.join(processed_dir, "train.bin")
test_out_file = os.path.join(processed_dir, "test.bin")
validation_out_file = os.path.join(processed_dir, "validation.bin")

In [36]:
write_to_bin(train,tokenized_stories_dir,train_out_file,processed_dir, makevocab=True)

Writing story 0 of 15; 0.0 percent done
Finished writing file //content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/dataFiles/test_processed/train.bin

Writing vocab file...
Finished writing vocab file


In [37]:
write_to_bin(test,tokenized_stories_dir,test_out_file,processed_dir, makevocab=False)

Writing story 0 of 9; 0.0 percent done
Finished writing file //content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/dataFiles/test_processed/test.bin



In [38]:
write_to_bin(val,tokenized_stories_dir,validation_out_file,processed_dir, makevocab=False)

Writing story 0 of 6; 0.0 percent done
Finished writing file //content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/dataFiles/test_processed/validation.bin



In [39]:
ls

test_processed/  test_tokenized/


In [40]:
pwd

'/content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/dataFiles'

In [41]:
cd test_processed/

/content/Drive/TS/TS/stanford-corenlp-full-2018-10-05/dataFiles/test_processed
